In [21]:
%load_ext autoreload
%autoreload 2
import logging
logging.basicConfig(level=logging.INFO)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
from src.tradecli import *
from src.journal import *

In [23]:
# Generate the feature dataframe
full_df

,trade_uid,type_1_m5,type_1_m15,type_2_w,type_1_m1,tp_price,entry_time,session,type_1_h1,type_3_m15,...,close_price,type_2_h1,type_2_m5,potential_return,initial_risk_reward,type_2_d,TP_distance,SL_distance,initial_return,sl_price
0,1,None,None,None,True,1.1100,2024-12-29 01:12:27.817885,tokyo,None,True,...,NaN,None,None,NaN,2.000000,None,0.0100,0.0050,0.0100,1.0950
1,2,None,None,None,None,1.2100,2024-12-29 01:12:27.818223,tokyo,None,None,...,1.1855,True,None,NaN,0.666667,None,0.0100,0.0150,0.0100,1.1850
2,3,None,None,None,None,1.3300,2024-12-27 10:00:01.000000,london,None,None,...,NaN,None,None,NaN,10.000000,None,0.0300,0.5000,0.0300,1.2970
3,4,None,None,None,None,1.3300,2024-12-25 10:00:00.000000,london,None,None,...,1.3050,None,None,0.0150,10.000000,None,0.0300,0.5000,0.0300,1.2970
4,5,None,None,None,True,1.1663,2024-12-29 01:12:27.819946,tokyo,None,None,...,1.1495,True,None,0.0299,5.000000,None,0.0285,0.0057,0.0285,1.1321
5,6,None,None,None,None,1.1810,2024-12-29 01:12:27.820287,tokyo,None,None,...,1.1511,None,None,0.0123,5.913580,True,0.0479,0.0081,0.0479,1.1250
6,7,True,None,None,None,1.1595,2024-12-29 01:12:27.820687,tokyo,None,None,...,NaN,None,None,0.0491,7.482759,True,0.0434,0.0058,0.0434,1.1103
7,8,None,True,None,None,1.1300,2024-12-29 01:12:27.821447,tokyo,None,None,...,NaN,None,None,0.0210,2.215190,True,0.0175,0.0079,0.0175,1.1046
8,9,None,None,None,None,1.1857,2024-12-29 01:12:27.821813,tokyo,None,None,...,1.1319,None,True,0.0232,3.978022,None,0.0362,0.0091,0.0362,1.1404
9,10,None,True,None,None,1.1559,2024-12-29 01:12:27.822595,tokyo,None,None,...,1.1386,None,None,0.0128,1.348315,None,0.0120,0.0089,0.0120,1.1350


In [24]:
!rm -rf output
j.to_markdown(output_dir='output')

2024-12-29 01:13:08 - root - INFO - Converting trades to markdown
2024-12-29 01:13:08 - root - INFO - Writing index markdown
2024-12-29 01:13:08 - root - INFO - Calculating simple statistics
2024-12-29 01:13:08 - root - INFO - Analyzing tag relevance
2024-12-29 01:13:08 - root - INFO - Finding best single tags and subsets
2024-12-29 01:13:08 - root - INFO - Finding best single tags
2024-12-29 01:13:08 - root - INFO - Finding best tag subsets
2024-12-29 01:13:08 - root - INFO - Finding best tag subsets
2024-12-29 01:13:09 - root - INFO - Calculating tags statistics
2024-12-29 01:13:09 - root - INFO - Plotting statistics
